In [1]:
##Liberias

# Generales
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import glob

# limpieza y stopwords
import re
from nltk.corpus import stopwords

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

## 1. Datos

In [2]:
#extrayendo data que proviene de 1. Cleaning-Resoluciones-Completas

DF = pd.read_csv("Total-data-resol-1y2Quinquenio.csv")
DF

,Número de resolución,Criterios aplicables,Año,Categoría,Materia,Empresa denunciada,Medios probatoriosCargo,Medios probatorios Descargo,Cuestiones,Normativa sectorial aplicada,Normativa específica,Otras normas,Multas,Medidas correctivas,Configuración del Tribunal,Decisión
0,0064-2006/TDC-INDECOPI,la responsabilidad administrativa de Transport...,2006,Pérdida de equipaje,Idoneidad del servicio,Empresa de transportes Flores Hermanos SRL,Boleta por servicio de encomienda,Denuncia policial por hurto,Competencia Fuerza mayor,Reglamento General de Ferrocarriles D. Supremo...,"DECRETO LEGISLATIVO N° 807 Facultades, normas ...",Código Civil,"0,50 UIT Pago de costas y costos",Pago de S/.850,Juan Francisco Rojas Leo Julio Durand Carrión ...,1
1,0071-2006/TDC-INDECOPI,El punto de partida para demostrar la existenc...,2006,Pérdida de equipaje,Idoneidad del servicio,Paredes Estrella Cargo SA (Estrella Cargo),0,0,"Competencia Fuerza mayor, caso fortuito",Ley General de Transporte y Tránsito Terrestre,"DECRETO LEGISLATIVO N° 807 Facultades, normas ...",Código Civil Sentencia del 23 de abril de 2007...,"0,5 UIT","Pago de S/.1,650",Juan Francisco Rojas Leo Julio Durand Carrión ...,1
2,0142-2006/TDC-INDECOPI,"Para el dictado de medidas correctivas, la aus...",2006,Pérdida de equipaje,Idoneidad del servicio Gradación de la sanción,Empresa de Tranporte Santa Úrsula (Transporte ...,0,0,0,Reglamento General de Ferrocarriles D. Supremo...,Ley No. 27917 Modifica el Texto Unico Ordenado...,Código Civil,"0,50 UIT",Pago de S/.850,Juan Francisco Rojas Leo Julio Durand Carrión ...,0
3,1335-2006/TDC-INDECOPI,la causa no imputable debe ser entendida como ...,2006,Pérdida de equipaje,Idoneidad del servicio,Empresa de Transporte El Pino SAC (El Pino),Boleta de Venta,0,deber de custodia del equipaje,Ley de Aeronaútica Civil del Perú DECRETO LEGI...,Res. 0603-2002/TDC-INDECOPI,0,"0,5 UIT Pago de costas y costos",Pago de S/. 966,Juan Francisco Rojas Leo Julio Durand Carrión ...,1
4,1428-2006/TDC.INDECOPI,al contratar unservicio de transporte de carga...,2006,Pérdida de equipaje,Idoneidad del servicio,Empresa de Transportes de pasajeros y carga Ca...,0,Guia Boleta de Venta No. 51285,0,D. Supremo No. 009-2004-MTC Reglamento Naciona...,"DECRETO LEGISLATIVO N° 807 Facultades, normas ...",Ley No. 27444 Ley del Procedimiento Administra...,0,0,Juan Francisco Rojas Leo Julio Durand Carrión ...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,1164-2015/SPC-INDECOPI,Como se ha desarrollado de manera clara en la ...,2015,Pérdida de equipaje,IDONEIDAD DEL SERVICIO,EMPRESA DE TRANSPORTES RONCO PERÚ S.A.C.,Boleto de Viaje N° 016939 Ticket de Equipaje N...,Denuncia policial,0,Código de Protección y Defensa del Consumidor,0,0,4 UIT,0,Julio Baltazar Durand Carrión Alejandro José R...,1
241,1208-2015/SPC-INDECOPI,"Por consiguiente, la presunta falta de atenció...",2015,1208-2015/SPC-INDECOPI,0,EMPRESA DE TRANSPORTE TURÍSTICO OLANO S.A.,0,0,0,Código de Protección y Defensa del Consumidor,Ley No. 27444 Ley del Procedimiento Administra...,0,0,0,Julio Baltazar Durand Carrión Alejandro José R...,1
242,1293-2015/SPC-INDECOPI,"En efecto, Transportes Yanatile alegó como sus...",2015,Pérdida de equipaje,RECURSO DE REVISIÓN,EMPRESA DE TRANSPORTES YANATILE S.R.L.,0,0,0,Código de Protección y Defensa del Consumidor,Ley No. 27444 Ley del Procedimiento Administra...,0,0,0,Julio Baltazar Durand Carrión Alejandro José R...,0
243,2688-2015/SPC-INDECOPI,"Asimismo, la Sala comparte los fundamentos de ...",2015,Pérdida de equipaje,IDONEIDAD DEL SERVICIO TRANSPORTE TERRESTRE,EMPRESA DE TRANSPORTE TURÍSTICO OLANO S.A.,0,0,0,Código de Protección y Defensa del Consumidor,Ley No. 27444 Ley del Procedimiento Administra...,0,0,0,Julio Baltazar Durand Carrión Alejandro José R...,1


In [3]:
DF["todo"] = DF['Número de resolución'] + " " + DF['Criterios aplicables'] + " " +str(DF['Año']) + " " +DF['Categoría']+ " " + DF["Materia"] + " " + DF["Empresa denunciada"] + " " + DF["Medios probatoriosCargo"] + " " + DF["Medios probatorios Descargo"] + " " + DF["Cuestiones"] + " " + DF["Normativa sectorial aplicada"] + " " + DF["Normativa específica"] + " " + DF["Otras normas"] + " " + DF["Multas"] + " " + DF["Medidas correctivas"] + " " + DF["Configuración del Tribunal"]

In [4]:
#Para este análisis solo usaremos Criterios aplicables y Decision

DF_nlp1 = DF[["Criterios aplicables", "Decisión"]]

In [5]:
DF_nlp2 = DF[["todo", "Decisión"]]

## 2. Limpieza

In [6]:
#visualizamos conteo

DF_nlp1["Decisión"].value_counts() #balanceado

1    136
0    109
Name: Decisión, dtype: int64

In [7]:
#quitar en NaaN
DF_nlp1.dropna(inplace=True)
DF_nlp2.dropna(inplace=True)

In [8]:
# quitamos caracteres especiales y numeros. Además quitamos las palabras pequeñas
# y un par de stopwords que no deberían estar (por obvias razones xD)

def limpieza(c):
    
    out = []
    c = re.findall(r"[a-zA-Z]+",c)
    for w in c:
        #aqui creo la lista
        if (len(w)>3 and (w!="para") and (w!="ademas") and (w!="culo")):
            out.append(w.lower())
    
    #aqui uno la lista en un cadena separada por espacios
    return " ".join(out)

In [9]:
# aplicar funcion de limpieza y creamos columna Limpio

DF_nlp1['Limpio']=DF_nlp1['Criterios aplicables'].astype("str").apply(limpieza)
DF_nlp2['Limpio']=DF_nlp2['todo'].astype("str").apply(limpieza)

In [10]:
#Quitamos stopwords predeterminadas de NLTK

def stopw(c):
    out=[]
    stop = stopwords.words("spanish") 
    for w in c.split():
        if w not in stop:
            out.append(w)
    return " ".join(out)

In [11]:
#aplicamos funcion de stopwords y creamos nueva columa SinSW

DF_nlp1['SinSW']=DF_nlp1['Limpio'].astype("str").apply(stopw)
DF_nlp2['SinSW']=DF_nlp2['Limpio'].astype("str").apply(stopw)

In [12]:
#creamos funcion para stemmer
from nltk.stem import SnowballStemmer

def Stemmer(c):
    out=[]
    spanish_stemmer = SnowballStemmer('spanish')
     
    for w in c.split():
        word_stem = spanish_stemmer.stem(str(w))
        out.append(word_stem)
    return " ".join(out)

In [13]:
#aplicamos funcion de stemmer
DF_nlp1['ConStem']=DF_nlp1['SinSW'].astype("str").apply(Stemmer)
DF_nlp2['ConStem']=DF_nlp2['SinSW'].astype("str").apply(Stemmer)

In [14]:
DF_nlp1

,Criterios aplicables,Decisión,Limpio,SinSW,ConStem
0,la responsabilidad administrativa de Transport...,1,responsabilidad administrativa transportes flo...,responsabilidad administrativa transportes flo...,respons administr transport flor laprestaci se...
1,El punto de partida para demostrar la existenc...,1,punto partida demostrar existencia causa exima...,punto partida demostrar existencia causa exima...,punt part demostr existent caus exim respons p...
2,"Para el dictado de medidas correctivas, la aus...",0,para dictado medidas correctivas ausencia regu...,dictado medidas correctivas ausencia regulaci ...,dict med correct ausenci regulaci espec ficaqu...
3,la causa no imputable debe ser entendida como ...,1,causa imputable debe entendida como evento ext...,causa imputable debe entendida evento extra es...,caus imput deb entend event extra esfer contro...
4,al contratar unservicio de transporte de carga...,0,contratar unservicio transporte carga consumid...,contratar unservicio transporte carga consumid...,contrat unservici transport carg consumidor ra...
...,...,...,...,...,...
240,Como se ha desarrollado de manera clara en la ...,1,como desarrollado manera clara presente resolu...,desarrollado manera clara presente resoluci re...,desarroll maner clar present resoluci reiter j...
241,"Por consiguiente, la presunta falta de atenció...",1,consiguiente presunta falta atenci carta notar...,consiguiente presunta falta atenci carta notar...,consiguient presunt falt atenci cart notarial ...
242,"En efecto, Transportes Yanatile alegó como sus...",0,efecto transportes yanatile aleg como sustento...,efecto transportes yanatile aleg sustento recu...,efect transport yanatil aleg sustent recurs re...
243,"Asimismo, la Sala comparte los fundamentos de ...",1,asimismo sala comparte fundamentos primera ins...,asimismo sala comparte fundamentos primera ins...,asim sal compart fundament primer instanci hec...


In [15]:
DF_nlp2

,todo,Decisión,Limpio,SinSW,ConStem
0,0064-2006/TDC-INDECOPI la responsabilidad admi...,1,indecopi responsabilidad administrativa transp...,indecopi responsabilidad administrativa transp...,indecopi respons administr transport flor lapr...
1,0071-2006/TDC-INDECOPI El punto de partida par...,1,indecopi punto partida demostrar existencia ca...,indecopi punto partida demostrar existencia ca...,indecopi punt part demostr existent caus exim ...
2,0142-2006/TDC-INDECOPI Para el dictado de medi...,0,indecopi para dictado medidas correctivas ause...,indecopi dictado medidas correctivas ausencia ...,indecopi dict med correct ausenci regulaci esp...
3,1335-2006/TDC-INDECOPI la causa no imputable d...,1,indecopi causa imputable debe entendida como e...,indecopi causa imputable debe entendida evento...,indecopi caus imput deb entend event extra esf...
4,1428-2006/TDC.INDECOPI al contratar unservicio...,0,indecopi contratar unservicio transporte carga...,indecopi contratar unservicio transporte carga...,indecopi contrat unservici transport carg cons...
...,...,...,...,...,...
240,1164-2015/SPC-INDECOPI Como se ha desarrollado...,1,indecopi como desarrollado manera clara presen...,indecopi desarrollado manera clara presente re...,indecopi desarroll maner clar present resoluci...
241,"1208-2015/SPC-INDECOPI Por consiguiente, la pr...",1,indecopi consiguiente presunta falta atenci ca...,indecopi consiguiente presunta falta atenci ca...,indecopi consiguient presunt falt atenci cart ...
242,"1293-2015/SPC-INDECOPI En efecto, Transportes ...",0,indecopi efecto transportes yanatile aleg como...,indecopi efecto transportes yanatile aleg sust...,indecopi efect transport yanatil aleg sustent ...
243,"2688-2015/SPC-INDECOPI Asimismo, la Sala compa...",1,indecopi asimismo sala comparte fundamentos pr...,indecopi asimismo sala comparte fundamentos pr...,indecopi asim sal compart fundament primer ins...


In [16]:
DF_nlp1.to_csv("Limpio-criterio.csv")

In [17]:
DF_nlp2.to_csv("Limpio-todo.csv")

## 3. Vectorizacion

### 3.1. TF

### 3.2. TF-IDF

In [18]:
criterios1 = DF_nlp1['ConStem'].values.tolist()
criterios2 = DF_nlp2['ConStem'].values.tolist()

In [19]:
#aplicamos tf-idf. Aquí se tienen solo 100 palabras,
# esto fue de manera experimental, se probo con 20, 50, 100 y se comparaba el aumento de acc de SVM
vectorizer1 = TfidfVectorizer(max_features=100)
FV1 = vectorizer1.fit_transform(criterios1)
FV1 = FV1.toarray()

In [20]:
vectorizer2 = TfidfVectorizer(max_features=100)
FV2 = vectorizer2.fit_transform(criterios2)
FV2 = FV2.toarray()

In [21]:
#mostramos palabras mas usadas
corpus1 = vectorizer1.get_feature_names()
corpus1

['acredit',
 'adecu',
 'administr',
 'alad',
 'aplic',
 'bien',
 'calid',
 'carg',
 'cas',
 'caus',
 'comisi',
 'condicion',
 'conduct',
 'confirm',
 'conform',
 'consecuent',
 'consider',
 'consumidor',
 'conten',
 'contrat',
 'correct',
 'correspond',
 'criteri',
 'cuent',
 'cumpl',
 'custodi',
 'deb',
 'decl',
 'decret',
 'defect',
 'denunci',
 'destin',
 'determin',
 'dich',
 'efect',
 'ello',
 'embarg',
 'empres',
 'encomiend',
 'entreg',
 'equipaj',
 'establec',
 'evit',
 'exim',
 'existent',
 'expuest',
 'extrav',
 'extravi',
 'extrem',
 'final',
 'hab',
 'hech',
 'idon',
 'impon',
 'indecopi',
 'infracci',
 'infractor',
 'legisl',
 'materi',
 'mayor',
 'med',
 'medi',
 'merc',
 'mism',
 'mont',
 'mult',
 'necesari',
 'norm',
 'objet',
 'obligaci',
 'ofrec',
 'orden',
 'part',
 'pasajer',
 'pes',
 'present',
 'prest',
 'prestaci',
 'proced',
 'product',
 'protecci',
 'proveedor',
 'pued',
 'rdid',
 'realiz',
 'resoluci',
 'respect',
 'respons',
 'result',
 'revert',
 'sal',
 'sa

In [22]:
corpus2 = vectorizer2.get_feature_names()
corpus2

['acredit',
 'adecu',
 'administr',
 'alberti',
 'bien',
 'bolet',
 'calid',
 'camil',
 'carg',
 'carrill',
 'cas',
 'civil',
 'comisi',
 'conduct',
 'confirm',
 'conform',
 'consecuent',
 'consider',
 'consumidor',
 'conten',
 'contrat',
 'correct',
 'correspond',
 'cost',
 'cuent',
 'cumpl',
 'custodi',
 'deb',
 'decret',
 'defect',
 'denunci',
 'destin',
 'determin',
 'dich',
 'dig',
 'dtype',
 'efect',
 'ello',
 'empres',
 'encomiend',
 'entreg',
 'equipaj',
 'establec',
 'existent',
 'extrav',
 'extrem',
 'facultad',
 'final',
 'francisc',
 'general',
 'hech',
 'hern',
 'idon',
 'impon',
 'indecopi',
 'infracci',
 'infractor',
 'juan',
 'legisl',
 'length',
 'luis',
 'materi',
 'med',
 'merc',
 'mism',
 'montoy',
 'nacional',
 'nam',
 'nicanor',
 'norm',
 'objet',
 'orden',
 'organizaci',
 'pag',
 'part',
 'pasajer',
 'pes',
 'present',
 'prest',
 'prestaci',
 'proced',
 'product',
 'protecci',
 'proveedor',
 'pued',
 'rdid',
 'reglament',
 'resoluci',
 'respect',
 'respons',
 're

In [23]:
#creamos columna de vectores caracte
DF_nlp1['FV'] = FV1.tolist()
DF_nlp1.head()

,Criterios aplicables,Decisión,Limpio,SinSW,ConStem,FV
0,la responsabilidad administrativa de Transport...,1,responsabilidad administrativa transportes flo...,responsabilidad administrativa transportes flo...,respons administr transport flor laprestaci se...,"[0.12690782898855163, 0.0, 0.07666692646210482..."
1,El punto de partida para demostrar la existenc...,1,punto partida demostrar existencia causa exima...,punto partida demostrar existencia causa exima...,punt part demostr existent caus exim respons p...,"[0.23452532302538726, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,"Para el dictado de medidas correctivas, la aus...",0,para dictado medidas correctivas ausencia regu...,dictado medidas correctivas ausencia regulaci ...,dict med correct ausenci regulaci espec ficaqu...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.23870759208102507,..."
3,la causa no imputable debe ser entendida como ...,1,causa imputable debe entendida como evento ext...,causa imputable debe entendida evento extra es...,caus imput deb entend event extra esfer contro...,"[0.0, 0.08634310097847096, 0.06669054072911727..."
4,al contratar unservicio de transporte de carga...,0,contratar unservicio transporte carga consumid...,contratar unservicio transporte carga consumid...,contrat unservici transport carg consumidor ra...,"[0.13867183116858692, 0.0, 0.167547473899763, ..."


In [24]:
DF_nlp2['FV'] = FV2.tolist()
DF_nlp2.head()

,todo,Decisión,Limpio,SinSW,ConStem,FV
0,0064-2006/TDC-INDECOPI la responsabilidad admi...,1,indecopi responsabilidad administrativa transp...,indecopi responsabilidad administrativa transp...,indecopi respons administr transport flor lapr...,"[0.11004376010471337, 0.0, 0.04538661625378674..."
1,0071-2006/TDC-INDECOPI El punto de partida par...,1,indecopi punto partida demostrar existencia ca...,indecopi punto partida demostrar existencia ca...,indecopi punt part demostr existent caus exim ...,"[0.2397600253772149, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0142-2006/TDC-INDECOPI Para el dictado de medi...,0,indecopi para dictado medidas correctivas ause...,indecopi dictado medidas correctivas ausencia ...,indecopi dict med correct ausenci regulaci esp...,"[0.0, 0.0, 0.0, 0.0, 0.13511273759830278, 0.0,..."
3,1335-2006/TDC-INDECOPI la causa no imputable d...,1,indecopi causa imputable debe entendida como e...,indecopi causa imputable debe entendida evento...,indecopi caus imput deb entend event extra esf...,"[0.0, 0.08969540942352676, 0.04762100128964493..."
4,1428-2006/TDC.INDECOPI al contratar unservicio...,0,indecopi contratar unservicio transporte carga...,indecopi contratar unservicio transporte carga...,indecopi contrat unservici transport carg cons...,"[0.12556457824941225, 0.0, 0.15536413848413808..."


## 4. Clasificación

#### Separando Train/Test (80%/20%)

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
#Entrada: vector característico
X1 = np.array(DF_nlp1.FV.to_list())

#Salida: Decision - 3 categorias
Y1 = np.array(DF_nlp1.Decisión.to_list())

In [27]:
X1.shape,Y1.shape

((245, 100), (245,))

In [28]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,
                                                    Y1,
                                                    test_size=0.2,
                                                   random_state=10)

In [29]:
X1_train.shape,X1_test.shape,y1_train.shape,y1_test.shape

((196, 100), (49, 100), (196,), (49,))

In [30]:
#Entrada: vector característico
X2 = np.array(DF_nlp2.FV.to_list())

#Salida: Decision - 3 categorias
Y2 = np.array(DF_nlp2.Decisión.to_list())

In [31]:
X2.shape,Y2.shape

((243, 100), (243,))

In [32]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2,
                                                    Y2,
                                                    test_size=0.2,
                                                   random_state=10)

In [33]:
X2_train.shape,X2_test.shape,y2_train.shape,y2_test.shape

((194, 100), (49, 100), (194,), (49,))

### 4.1. SVM

In [34]:
from sklearn import svm

In [35]:
clasificador1 = svm.SVC(kernel='rbf')
clasificador1.fit(X1_train,y1_train)

SVC()

In [36]:
Y1_pred = clasificador1.predict(X1_test)

In [37]:
Y1_pred.shape

(49,)

In [38]:
from sklearn.metrics import accuracy_score,classification_report

In [39]:
accuracy_score(y1_test,Y1_pred)

0.7755102040816326

In [40]:
print(classification_report(y1_test,Y1_pred))

              precision    recall  f1-score   support

           0       0.82      0.64      0.72        22
           1       0.75      0.89      0.81        27

    accuracy                           0.78        49
   macro avg       0.79      0.76      0.77        49
weighted avg       0.78      0.78      0.77        49



In [41]:
clasificador2 = svm.SVC(kernel='rbf')
clasificador2.fit(X2_train,y2_train)

SVC()

In [42]:
Y2_pred = clasificador2.predict(X2_test)

In [43]:
Y2_pred.shape

(49,)

In [44]:
from sklearn.metrics import accuracy_score,classification_report

In [45]:
accuracy_score(y2_test,Y2_pred)

0.7959183673469388

In [46]:
print(classification_report(y2_test,Y2_pred))

              precision    recall  f1-score   support

           0       0.75      0.67      0.71        18
           1       0.82      0.87      0.84        31

    accuracy                           0.80        49
   macro avg       0.78      0.77      0.77        49
weighted avg       0.79      0.80      0.79        49



### 4.2 MLP

In [47]:
import tensorflow as tf
from keras import layers
import keras

In [48]:
### MLP

modeloMLP1 = tf.keras.Sequential()

#MLP 3 capas ocultas
modeloMLP1.add(layers.Dense(10,activation='relu'))

#salida
modeloMLP1.add(layers.Dense(units=1, activation='sigmoid'))

In [49]:
modeloMLP1.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [50]:
modeloMLP1.fit(X1_train, y1_train,epochs=100)

Epoch 1/100
7/7 [==============================] - 0s 770us/step - loss: 0.7052 - accuracy: 0.4745
Epoch 2/100
7/7 [==============================] - 0s 698us/step - loss: 0.6943 - accuracy: 0.5408
Epoch 3/100
7/7 [==============================] - 0s 703us/step - loss: 0.6869 - accuracy: 0.5714
Epoch 4/100
7/7 [==============================] - 0s 700us/step - loss: 0.6814 - accuracy: 0.6429
Epoch 5/100
7/7 [==============================] - 0s 785us/step - loss: 0.6761 - accuracy: 0.6429
Epoch 6/100
7/7 [==============================] - 0s 999us/step - loss: 0.6707 - accuracy: 0.6480
Epoch 7/100
7/7 [==============================] - 0s 715us/step - loss: 0.6653 - accuracy: 0.6480
Epoch 8/100
7/7 [==============================] - 0s 658us/step - loss: 0.6603 - accuracy: 0.6480
Epoch 9/100
7/7 [==============================] - 0s 1ms/step - loss: 0.6554 - accuracy: 0.6735
Epoch 10/100
7/7 [==============================] - 0s 1ms/step - loss: 0.6508 - accuracy: 0.6684
Epoch 11/100


7/7 [==============================] - 0s 914us/step - loss: 0.3789 - accuracy: 0.8571
Epoch 84/100
7/7 [==============================] - 0s 1ms/step - loss: 0.3768 - accuracy: 0.8520
Epoch 85/100
7/7 [==============================] - 0s 809us/step - loss: 0.3749 - accuracy: 0.8520
Epoch 86/100
7/7 [==============================] - 0s 849us/step - loss: 0.3729 - accuracy: 0.8571
Epoch 87/100
7/7 [==============================] - 0s 856us/step - loss: 0.3707 - accuracy: 0.8571
Epoch 88/100
7/7 [==============================] - 0s 985us/step - loss: 0.3688 - accuracy: 0.8520
Epoch 89/100
7/7 [==============================] - 0s 933us/step - loss: 0.3670 - accuracy: 0.8622
Epoch 90/100
7/7 [==============================] - 0s 683us/step - loss: 0.3649 - accuracy: 0.8622
Epoch 91/100
7/7 [==============================] - 0s 1ms/step - loss: 0.3626 - accuracy: 0.8622
Epoch 92/100
7/7 [==============================] - 0s 803us/step - loss: 0.3606 - accuracy: 0.8622
Epoch 93/100
7/7 

In [51]:
# hacer clasification report

In [52]:
Y1_pred_MLP_eval = modeloMLP1.evaluate(X1_test,y1_test) #:p

2/2 [==============================] - 0s 782us/step - loss: 0.5935 - accuracy: 0.7143


In [53]:
Y1_pred_MLP = modeloMLP1.predict(X1_test)

In [54]:
### MLP 2 PARA TODO

modeloMLP2 = tf.keras.Sequential()

#MLP 3 capas ocultas
modeloMLP2.add(layers.Dense(7,activation='relu'))

#salida
modeloMLP2.add(layers.Dense(units=1, activation='sigmoid'))

In [55]:
modeloMLP2.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [61]:
modeloMLP2.fit(X2_train, y2_train,epochs=100)

Epoch 1/100
7/7 [==============================] - 0s 749us/step - loss: 0.4451 - accuracy: 0.8299
Epoch 2/100
7/7 [==============================] - 0s 777us/step - loss: 0.4434 - accuracy: 0.8247
Epoch 3/100
7/7 [==============================] - 0s 700us/step - loss: 0.4422 - accuracy: 0.8247
Epoch 4/100
7/7 [==============================] - 0s 729us/step - loss: 0.4405 - accuracy: 0.8351
Epoch 5/100
7/7 [==============================] - 0s 715us/step - loss: 0.4385 - accuracy: 0.8402
Epoch 6/100
7/7 [==============================] - 0s 707us/step - loss: 0.4376 - accuracy: 0.8402
Epoch 7/100
7/7 [==============================] - 0s 703us/step - loss: 0.4359 - accuracy: 0.8402
Epoch 8/100
7/7 [==============================] - 0s 667us/step - loss: 0.4348 - accuracy: 0.8454
Epoch 9/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4333 - accuracy: 0.8454
Epoch 10/100
7/7 [==============================] - 0s 761us/step - loss: 0.4321 - accuracy: 0.8505
Epoch 11/10

7/7 [==============================] - 0s 843us/step - loss: 0.3437 - accuracy: 0.8763
Epoch 84/100
7/7 [==============================] - 0s 675us/step - loss: 0.3427 - accuracy: 0.8763
Epoch 85/100
7/7 [==============================] - 0s 758us/step - loss: 0.3415 - accuracy: 0.8763
Epoch 86/100
7/7 [==============================] - 0s 678us/step - loss: 0.3406 - accuracy: 0.8763
Epoch 87/100
7/7 [==============================] - 0s 747us/step - loss: 0.3394 - accuracy: 0.8763
Epoch 88/100
7/7 [==============================] - 0s 1ms/step - loss: 0.3389 - accuracy: 0.8763
Epoch 89/100
7/7 [==============================] - 0s 964us/step - loss: 0.3385 - accuracy: 0.8763
Epoch 90/100
7/7 [==============================] - 0s 1ms/step - loss: 0.3378 - accuracy: 0.8763
Epoch 91/100
7/7 [==============================] - 0s 921us/step - loss: 0.3366 - accuracy: 0.8763
Epoch 92/100
7/7 [==============================] - 0s 1ms/step - loss: 0.3348 - accuracy: 0.8763
Epoch 93/100
7/7 [=

In [57]:
# hacer clasification report

In [62]:
Y2_pred_MLP_eval = modeloMLP2.evaluate(X2_test,y2_test) #:p

2/2 [==============================] - 0s 746us/step - loss: 0.6282 - accuracy: 0.6939


In [59]:
Y2_pred_MLP = modeloMLP2.predict(X2_test)